In [1]:
#!pip install textblob
#!pip install langdetect
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tianj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [1]:
#imports
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re
import random
from scipy import sparse
from scipy.sparse import csr_matrix, vstack
from textblob import TextBlob
from langdetect import detect_langs
import pickle
from datetime import datetime

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
import multiprocessing
import gensim
from gensim.utils import simple_preprocess

from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from langdetect import detect
# from wordcloud import WordCloud, STOPWORDS
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn import svm
import plotly.graph_objects as go
# from better_profanity import profanity
import string

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [2]:
songdata = pd.read_csv('./songdata.csv')

In [3]:
songdata.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df = pd.read_pickle("./genres.pkl")
df = pd.merge(songdata, df, on=['artist','artist'])
df = df[df.iloc[:,5:].any(axis = 1)]

In [6]:
df.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA...","[europop, swedish pop]",0,0,1,0,0,0,0
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...","[europop, swedish pop]",0,0,1,0,0,0,0
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...,"[europop, swedish pop]",0,0,1,0,0,0,0
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0


In [14]:
df.iloc[:,5:].sum()/df.iloc[:,5:].count()

rock                 0.750957
singer-songwriter    0.177727
pop                  0.450174
metal                0.150945
folk                 0.206316
country              0.182680
hip hop / rap        0.077026
dtype: float64

In [8]:
pd.DataFrame(pd.DataFrame(y_test_vec).sum()/len(pd.DataFrame(y_test_vec)))
    null.iloc[1:,0] = 1-null.iloc[1:,0]

IndentationError: unexpected indent (<ipython-input-8-f21ccf5f8755>, line 2)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51489 entries, 0 to 57595
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   artist             51489 non-null  object
 1   song               51489 non-null  object
 2   link               51489 non-null  object
 3   text               51489 non-null  object
 4   genres             51489 non-null  object
 5   rock               51489 non-null  int32 
 6   singer-songwriter  51489 non-null  int32 
 7   pop                51489 non-null  int32 
 8   metal              51489 non-null  int32 
 9   folk               51489 non-null  int32 
 10  country            51489 non-null  int32 
 11  hip hop / rap      51489 non-null  int32 
dtypes: int32(7), object(5)
memory usage: 3.7+ MB


We have a multilabel classification problem according to https://scikit-learn.org/stable/modules/multiclass.html

Multilabel- 

"A multiclass multioutput target where each output is binary. This may be represented as a 2d (dense) array or sparse matrix of integers, such that each column is a separate binary target, where positive labels are indicated with 1 and negative labels are usually -1 or 0. Sparse multilabel targets are not supported everywhere that dense multilabel targets are supported."

"Valid representation of multilabel y is either dense (or sparse) binary matrix of shape (n_samples, n_classes). Each column represents a class. The 1’s in each row denote the positive classes a sample has been labelled with."

Some text is in round brackets while some is in square brackets. I wanted to examine what that text looked like.

## Removing Non-English Words

### Removing Non-English Songs

Note: This takes a while to run. 

In [8]:
#This find the probability that the word is english
def get_eng_prob(text):
    detections = detect_langs(text)
    for detection in detections:
        if detection.lang == 'en':
            return detection.prob
    return 0

#finding the probability that the text is english
df['en_prob'] = df['text'].map(get_eng_prob)

print('Number of songs in english: {}'.format(sum(df['en_prob'] >= 0.5)))
print('Number of songs that are not english: {}'.format(sum(df['en_prob'] < 0.5)))

Number of songs in english: 51123
Number of songs that are not english: 366


In [9]:
#only selecting songs that have a probability of 0.5 or higher of being in english
df= df.loc[df['en_prob'] >= 0.5]

In [10]:
df.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,en_prob
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA...","[europop, swedish pop]",0,0,1,0,0,0,0,0.999996
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...","[europop, swedish pop]",0,0,1,0,0,0,0,0.999997
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...,"[europop, swedish pop]",0,0,1,0,0,0,0,0.999997
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,0.999998
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,0.999998


In [11]:
#Repeating above code to make sure that the non-english songs were dropped
print('Number of songs in english: {}'.format(sum(df['en_prob'] >= 0.5)))
print('Number of songs that are not english: {}'.format(sum(df['en_prob'] < 0.5)))

Number of songs in english: 51123
Number of songs that are not english: 0


In [12]:
df = df.drop(['en_prob'], axis=1)

## Splitting df into Train/Test
We split the initial dataframe into train/test sets before applying any text preprocessing. This is to avoid any unintentional influence of training sets on testing sets, as preprocessing tools performing imputations on the entire data set.

In [13]:
df_train, df_test = train_test_split(df, 
                                     test_size = 0.2,
                                     random_state = 0)

In [14]:
df_train.shape

(40898, 12)

In [15]:
df_test.shape

(10225, 12)

## Text PreProcessing - df Train

In [16]:
#examining text in round brackets
round_brackets = sum(list(df_train['text'].map(lambda s: re.findall(r'\((.*?)\)',s))), [])
#Number of round brackets:
print((len(round_brackets)))

43577


In [17]:
#Viewing some text in round brackets
#These just look like normal lyrics
random.seed(0)
random.choices(round_brackets, k=15)

['Verse',
 'repeat and fade',
 'repeat chorus',
 'you were right girl',
 'oh oh no, oh oh no',
 'you will tell me',
 "when we ride we ride it's til the day we die",
 'no',
 'This is love',
 'Clap, clap, clap',
 'how are you',
 'yeah, yeah, yeah',
 'I need all my cash right now',
 'Oohhh baby',
 'Gonna make you mine']

In [18]:
#examining text in square brackets
square_brackets = sum(list(df_train['text'].map(lambda s: re.findall(r'\[(.*?)\]',s))), [])
#how many instances of square brackets?
print((len(square_brackets)))

20645


In [19]:
#re.sub(pattern, repl, string, count=0, flags=0)
# remove round brackets but not text within
df_train['text'] = df_train['text'].map(lambda s: re.sub(r'\(|\)', '', s))

# remove square brackest and text within
df_train['text'] = df_train['text'].map(lambda s: re.sub(r'\[(.*?)\] ', '', s))

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
# remove line breaks
df_train['text'] = df_train['text'].map(lambda s: re.sub(r' \n|\n', '', s))

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
df_train['text'].map(lambda s: re.sub(r' \n|\n', '', s))

40373    My daddy was the family bassman  \rMy mamma wa...
41147    Thick cloud, steam rising, hissing stone on sw...
26951    Somewhere tonight, there's somebody cryin'  \r...
32586    I've found the center of fruit as late  \rIs t...
28650    How many times have they lied with the truth i...
                               ...                        
24081    She was just a country girl  \rDidn't have tho...
51806    There was a time when men would prosper  \rThe...
47909    Doesn't she look so well  \rWith her long blac...
49069    Tell me a story of mind over matter  \rThe hop...
2961     Hot fun in the summertime  \ryeah bow bow bow ...
Name: text, Length: 40898, dtype: object

### Stemming and Preprocessing

In [22]:
stemmer = PorterStemmer()
def lemmatize_stemming(text):
 
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):

    result = []
    #stopwords = list(gensim.parsing.preprocessing.STOPWORDS)
    #stopwords.pop(stopwords.index('no'))
    for token in gensim.utils.simple_preprocess(text):
        #if token not in set(stopwords):
        result.append(lemmatize_stemming(token))
    return ' '.join(result)

In [23]:
df_train['text_preprocessed'] = df_train['text'].map(preprocess)

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
df_train.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed
40373,Paul Simon,Baby Driver,/p/paul+simon/baby+driver_20151462.html,My daddy was the family bassman \rMy mamma wa...,"[classic rock, folk, folk rock, mellow gold, p...",1,1,0,0,1,0,0,my daddi be the famili bassman my mamma be an ...
41147,Peter Gabriel,San Jacinto,/p/peter+gabriel/san+jacinto_10204715.html,"Thick cloud, steam rising, hissing stone on sw...","[album rock, art pop, art rock, classic rock, ...",1,0,1,0,0,0,0,thick cloud steam rise hiss stone on sweat lod...
26951,Keith Urban,Sometimes Angels Can't Fly,/k/keith+urban/sometimes+angels+cant+fly_20571...,"Somewhere tonight, there's somebody cryin' \r...","[australian country, australian pop, contempor...",0,0,1,0,0,1,0,somewher tonight there somebodi cryin and feel...
32586,Marilyn Manson,Apple Of Sodom,/m/marilyn+manson/apple+of+sodom_20089096.html,I've found the center of fruit as late \rIs t...,"[alternative metal, industrial, industrial met...",1,0,0,1,0,0,0,ve find the center of fruit as late be the cen...
28650,Kiss,I've Had Enough,/k/kiss/ive+had+enough_10159193.html,How many times have they lied with the truth i...,"[album rock, classic rock, glam rock, hard roc...",1,0,0,1,0,0,0,how mani time have they lie with the truth in ...


In [25]:
#Checking what the tokens column look like before removing stopwords
df_train['text_preprocessed'].head()

40373    my daddi be the famili bassman my mamma be an ...
41147    thick cloud steam rise hiss stone on sweat lod...
26951    somewher tonight there somebodi cryin and feel...
32586    ve find the center of fruit as late be the cen...
28650    how mani time have they lie with the truth in ...
Name: text_preprocessed, dtype: object

In [26]:
#safety
df_train2 = df_train

## Tokenization

In [27]:
#w+ for whitespace
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

#creating a new column called tokens
df_train['tokens'] = df_train['text_preprocessed'].map(tokenizer.tokenize)

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [28]:
df_train.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens
40373,Paul Simon,Baby Driver,/p/paul+simon/baby+driver_20151462.html,My daddy was the family bassman \rMy mamma wa...,"[classic rock, folk, folk rock, mellow gold, p...",1,1,0,0,1,0,0,my daddi be the famili bassman my mamma be an ...,"[my, daddi, be, the, famili, bassman, my, mamm..."
41147,Peter Gabriel,San Jacinto,/p/peter+gabriel/san+jacinto_10204715.html,"Thick cloud, steam rising, hissing stone on sw...","[album rock, art pop, art rock, classic rock, ...",1,0,1,0,0,0,0,thick cloud steam rise hiss stone on sweat lod...,"[thick, cloud, steam, rise, hiss, stone, on, s..."
26951,Keith Urban,Sometimes Angels Can't Fly,/k/keith+urban/sometimes+angels+cant+fly_20571...,"Somewhere tonight, there's somebody cryin' \r...","[australian country, australian pop, contempor...",0,0,1,0,0,1,0,somewher tonight there somebodi cryin and feel...,"[somewher, tonight, there, somebodi, cryin, an..."
32586,Marilyn Manson,Apple Of Sodom,/m/marilyn+manson/apple+of+sodom_20089096.html,I've found the center of fruit as late \rIs t...,"[alternative metal, industrial, industrial met...",1,0,0,1,0,0,0,ve find the center of fruit as late be the cen...,"[ve, find, the, center, of, fruit, as, late, b..."
28650,Kiss,I've Had Enough,/k/kiss/ive+had+enough_10159193.html,How many times have they lied with the truth i...,"[album rock, classic rock, glam rock, hard roc...",1,0,0,1,0,0,0,how mani time have they lie with the truth in ...,"[how, mani, time, have, they, lie, with, the, ..."


### Removing stop words
Reference for more info: https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

In [29]:
#setting stop words
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tianj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
#removing stop words
df_train['tokens'] = df_train['tokens'].apply(lambda x: [item for item in x if item not in stop_words])

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [31]:
df_train.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens
40373,Paul Simon,Baby Driver,/p/paul+simon/baby+driver_20151462.html,My daddy was the family bassman \rMy mamma wa...,"[classic rock, folk, folk rock, mellow gold, p...",1,1,0,0,1,0,0,my daddi be the famili bassman my mamma be an ...,"[daddi, famili, bassman, mamma, engin, bear, o..."
41147,Peter Gabriel,San Jacinto,/p/peter+gabriel/san+jacinto_10204715.html,"Thick cloud, steam rising, hissing stone on sw...","[album rock, art pop, art rock, classic rock, ...",1,0,1,0,0,0,0,thick cloud steam rise hiss stone on sweat lod...,"[thick, cloud, steam, rise, hiss, stone, sweat..."
26951,Keith Urban,Sometimes Angels Can't Fly,/k/keith+urban/sometimes+angels+cant+fly_20571...,"Somewhere tonight, there's somebody cryin' \r...","[australian country, australian pop, contempor...",0,0,1,0,0,1,0,somewher tonight there somebodi cryin and feel...,"[somewher, tonight, somebodi, cryin, feelin, c..."
32586,Marilyn Manson,Apple Of Sodom,/m/marilyn+manson/apple+of+sodom_20089096.html,I've found the center of fruit as late \rIs t...,"[alternative metal, industrial, industrial met...",1,0,0,1,0,0,0,ve find the center of fruit as late be the cen...,"[find, center, fruit, late, center, truth, tod..."
28650,Kiss,I've Had Enough,/k/kiss/ive+had+enough_10159193.html,How many times have they lied with the truth i...,"[album rock, classic rock, glam rock, hard roc...",1,0,0,1,0,0,0,how mani time have they lie with the truth in ...,"[mani, time, lie, truth, eye, treat, like, dir..."


In [32]:
#checking out the data
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40898 entries, 40373 to 2961
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   artist             40898 non-null  object
 1   song               40898 non-null  object
 2   link               40898 non-null  object
 3   text               40898 non-null  object
 4   genres             40898 non-null  object
 5   rock               40898 non-null  int32 
 6   singer-songwriter  40898 non-null  int32 
 7   pop                40898 non-null  int32 
 8   metal              40898 non-null  int32 
 9   folk               40898 non-null  int32 
 10  country            40898 non-null  int32 
 11  hip hop / rap      40898 non-null  int32 
 12  text_preprocessed  40898 non-null  object
 13  tokens             40898 non-null  object
dtypes: int32(7), object(7)
memory usage: 3.6+ MB


In [33]:
#turning the stems back into untokenized structrure so that we can run count vector on it
df_train['stem_str'] = df_train['tokens'].map(lambda lst: ' '.join(lst))

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [34]:
df_train.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens,stem_str
40373,Paul Simon,Baby Driver,/p/paul+simon/baby+driver_20151462.html,My daddy was the family bassman \rMy mamma wa...,"[classic rock, folk, folk rock, mellow gold, p...",1,1,0,0,1,0,0,my daddi be the famili bassman my mamma be an ...,"[daddi, famili, bassman, mamma, engin, bear, o...",daddi famili bassman mamma engin bear one dark...
41147,Peter Gabriel,San Jacinto,/p/peter+gabriel/san+jacinto_10204715.html,"Thick cloud, steam rising, hissing stone on sw...","[album rock, art pop, art rock, classic rock, ...",1,0,1,0,0,0,0,thick cloud steam rise hiss stone on sweat lod...,"[thick, cloud, steam, rise, hiss, stone, sweat...",thick cloud steam rise hiss stone sweat lodg f...
26951,Keith Urban,Sometimes Angels Can't Fly,/k/keith+urban/sometimes+angels+cant+fly_20571...,"Somewhere tonight, there's somebody cryin' \r...","[australian country, australian pop, contempor...",0,0,1,0,0,1,0,somewher tonight there somebodi cryin and feel...,"[somewher, tonight, somebodi, cryin, feelin, c...",somewher tonight somebodi cryin feelin complet...
32586,Marilyn Manson,Apple Of Sodom,/m/marilyn+manson/apple+of+sodom_20089096.html,I've found the center of fruit as late \rIs t...,"[alternative metal, industrial, industrial met...",1,0,0,1,0,0,0,ve find the center of fruit as late be the cen...,"[find, center, fruit, late, center, truth, tod...",find center fruit late center truth today cut ...
28650,Kiss,I've Had Enough,/k/kiss/ive+had+enough_10159193.html,How many times have they lied with the truth i...,"[album rock, classic rock, glam rock, hard roc...",1,0,0,1,0,0,0,how mani time have they lie with the truth in ...,"[mani, time, lie, truth, eye, treat, like, dir...",mani time lie truth eye treat like dirt wast d...


### Vectorization

In [35]:
df_train['tokens2']=df_train.tokens.apply(lambda x:str(x).strip('[]'))
df_train.head()                  

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens,stem_str,tokens2
40373,Paul Simon,Baby Driver,/p/paul+simon/baby+driver_20151462.html,My daddy was the family bassman \rMy mamma wa...,"[classic rock, folk, folk rock, mellow gold, p...",1,1,0,0,1,0,0,my daddi be the famili bassman my mamma be an ...,"[daddi, famili, bassman, mamma, engin, bear, o...",daddi famili bassman mamma engin bear one dark...,"'daddi', 'famili', 'bassman', 'mamma', 'engin'..."
41147,Peter Gabriel,San Jacinto,/p/peter+gabriel/san+jacinto_10204715.html,"Thick cloud, steam rising, hissing stone on sw...","[album rock, art pop, art rock, classic rock, ...",1,0,1,0,0,0,0,thick cloud steam rise hiss stone on sweat lod...,"[thick, cloud, steam, rise, hiss, stone, sweat...",thick cloud steam rise hiss stone sweat lodg f...,"'thick', 'cloud', 'steam', 'rise', 'hiss', 'st..."
26951,Keith Urban,Sometimes Angels Can't Fly,/k/keith+urban/sometimes+angels+cant+fly_20571...,"Somewhere tonight, there's somebody cryin' \r...","[australian country, australian pop, contempor...",0,0,1,0,0,1,0,somewher tonight there somebodi cryin and feel...,"[somewher, tonight, somebodi, cryin, feelin, c...",somewher tonight somebodi cryin feelin complet...,"'somewher', 'tonight', 'somebodi', 'cryin', 'f..."
32586,Marilyn Manson,Apple Of Sodom,/m/marilyn+manson/apple+of+sodom_20089096.html,I've found the center of fruit as late \rIs t...,"[alternative metal, industrial, industrial met...",1,0,0,1,0,0,0,ve find the center of fruit as late be the cen...,"[find, center, fruit, late, center, truth, tod...",find center fruit late center truth today cut ...,"'find', 'center', 'fruit', 'late', 'center', '..."
28650,Kiss,I've Had Enough,/k/kiss/ive+had+enough_10159193.html,How many times have they lied with the truth i...,"[album rock, classic rock, glam rock, hard roc...",1,0,0,1,0,0,0,how mani time have they lie with the truth in ...,"[mani, time, lie, truth, eye, treat, like, dir...",mani time lie truth eye treat like dirt wast d...,"'mani', 'time', 'lie', 'truth', 'eye', 'treat'..."


we want to keep: 

artists

song title

tokens2 (name it lyrics)

genre x7

## Doc2Vec

In [36]:
train_tokens_documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df_train.tokens)]

In [37]:
train_tokens_documents[1]

TaggedDocument(words=['thick', 'cloud', 'steam', 'rise', 'hiss', 'stone', 'sweat', 'lodg', 'fire', 'around', 'buffalo', 'robe', 'sage', 'bundl', 'rub', 'skin', 'outsid', 'cold', 'air', 'stand', 'wait', 'rise', 'sun', 'red', 'paint', 'eagl', 'feather', 'coyot', 'call', 'begin', 'someth', 'move', 'tast', 'mouth', 'heart', 'feel', 'like', 'die', 'slow', 'let', 'go', 'life', 'medicin', 'man', 'lead', 'though', 'town', 'indian', 'grind', 'far', 'cut', 'land', 'hous', 'pool', 'kid', 'wear', 'water', 'wing', 'drink', 'cool', 'follow', 'dri', 'river', 'bed', 'watch', 'scout', 'guid', 'make', 'pow', 'wow', 'sign', 'past', 'geronimo', 'disco', 'sit', 'bull', 'steakhous', 'white', 'men', 'dream', 'rattl', 'old', 'man', 'sack', 'look', 'mountain', 'top', 'keep', 'climb', 'way', 'abov', 'us', 'desert', 'snow', 'white', 'wind', 'blow', 'hold', 'line', 'line', 'strength', 'pull', 'fear', 'san', 'jacinto', 'hold', 'line', 'san', 'jacinto', 'poison', 'bite', 'dark', 'take', 'sight', 'hold', 'line', 'te

In [38]:
model_train = gensim.models.doc2vec.Doc2Vec(train_tokens_documents,
                                            vector_size = 50, 
                                            min_count = 2, 
                                            epochs = 20) #Takes 5min

In [39]:
model_train.train(train_tokens_documents, 
                  total_examples = model_train.corpus_count, 
                  epochs = model_train.epochs) #Takes 5min

In [40]:
# used for running predictions on new data
model_train.infer_vector(train_tokens_documents[0].words, steps=20) #This changes every time. But within a certain range.

array([ 0.08475662,  1.3761533 , -0.5588185 ,  0.11752684,  2.2714608 ,
        0.0562108 , -0.00347753, -2.36126   ,  0.0293406 , -1.5405602 ,
        0.23659953, -1.6832412 , -0.01126645,  0.35125804,  0.4581896 ,
        0.75753623, -0.3104734 ,  0.03815316,  0.23960365,  1.1894764 ,
       -0.11772769,  0.57643664,  1.3587031 ,  0.9474954 ,  0.45312917,
       -1.1637137 , -0.2701957 ,  0.94534004, -0.21243228,  0.519806  ,
       -1.7390851 , -0.32897037,  1.4100754 , -0.7008101 ,  1.472671  ,
        0.27304086, -0.9464377 , -0.44091222,  0.8195579 ,  0.07797348,
       -0.69892615, -1.0870702 ,  0.7338403 , -1.444114  ,  0.04592748,
       -1.0244366 , -0.6079197 , -0.42977798, -1.0596353 ,  0.40613478],
      dtype=float32)

In [41]:
def vector_for_learning(model, input_docs):
    sents = input_docs
    targets, feature_vectors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, feature_vectors

In [42]:
target_train, veclyrics_doc2vec_train = vector_for_learning(model_train, train_tokens_documents) #Takes 5min

In [43]:
veclyrics_doc2vec_train[0]

array([-0.05485782,  1.2927572 , -0.6623983 , -0.07667152,  2.3892844 ,
        0.0284148 , -0.14886579, -2.3995256 , -0.10637041, -1.6963567 ,
        0.1137131 , -1.7074509 ,  0.24779586,  0.28374553,  0.18758109,
        0.7104693 , -0.35393214, -0.16427743,  0.39106247,  1.3456806 ,
       -0.11194604,  0.55751544,  1.0971428 ,  1.0146092 ,  0.47847277,
       -1.1692252 , -0.15796492,  0.9106517 , -0.1385818 ,  0.5738896 ,
       -1.5204576 , -0.4767453 ,  1.391366  , -0.7609993 ,  1.4421828 ,
        0.28374863, -0.7967517 , -0.17051506,  0.87783676,  0.14334075,
       -0.6873524 , -0.6699573 ,  0.8795034 , -1.5488001 ,  0.3306943 ,
       -1.1330358 , -0.5784432 , -0.39463815, -0.9420877 ,  0.40650827],
      dtype=float32)

In [44]:
type(veclyrics_doc2vec_train)

tuple

In [45]:
len(veclyrics_doc2vec_train)

40898

## Text PreProcess - df Test

In [46]:
#examining text in round brackets
round_brackets = sum(list(df_test['text'].map(lambda s: re.findall(r'\((.*?)\)',s))), [])
#Number of round brackets:
print((len(round_brackets)))

10986


In [47]:
#Viewing some text in round brackets
#These just look like normal lyrics
random.seed(0)
random.choices(round_brackets, k=15)

['around',
 'Oh!',
 'For your love',
 'talking',
 'I love a lonely day',
 "Dancin' in the streets",
 "Don't tease me baby",
 'ooh',
 "Daddy's little girl",
 'Have no fear',
 'gangsta',
 'I see Jesus standing waiting for me',
 'Love will mend your broken wing',
 "And I'm nervous",
 'you know I want ya babe']

In [48]:
#examining text in square brackets
square_brackets = sum(list(df_test['text'].map(lambda s: re.findall(r'\[(.*?)\]',s))), [])
#how many instances of square brackets?
print((len(square_brackets)))

5128


In [49]:
#re.sub(pattern, repl, string, count=0, flags=0)
# remove round brackets but not text within
df_test['text'] = df_test['text'].map(lambda s: re.sub(r'\(|\)', '', s))

# remove square brackest and text within
df_test['text'] = df_test['text'].map(lambda s: re.sub(r'\[(.*?)\] ', '', s))

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [50]:
# remove line breaks
df_test['text'] = df_test['text'].map(lambda s: re.sub(r' \n|\n', '', s))

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Stemming and Preprocessing

In [51]:
stemmer = PorterStemmer()
def lemmatize_stemming(text):
 
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):

    result = []
    #stopwords = list(gensim.parsing.preprocessing.STOPWORDS)
    #stopwords.pop(stopwords.index('no'))
    for token in gensim.utils.simple_preprocess(text):
        #if token not in set(stopwords):
        result.append(lemmatize_stemming(token))
    return ' '.join(result)

In [52]:
df_test['text_preprocessed'] = df_test['text'].map(preprocess)

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [53]:
df_test.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed
41483,Phish,Back On The Train,/p/phish/back+on+the+train_20152042.html,"When I jumped off, I had a bucket full of thou...","[blues rock, jam band, roots rock]",1,0,0,0,0,0,0,when jump off have bucket full of thought when...
26062,Judds,In My Dreams,/j/judds/in+my+dreams_20076332.html,In my dreams \rYou belong to me \rShe doesn'...,"[country, country dawn, country rock]",1,0,0,0,0,1,0,in my dream you belong to me she doesn mean th...
135,Ace Of Base,C'est La Vie,/a/ace+of+base/cest+la+vie_10002096.html,"I am as young as I'm feeling \rI'm always 21,...","[bubblegum dance, dance pop, eurodance, europo...",1,0,1,0,0,0,0,be as young as feel alway be alway est la vie ...
18079,Glee,Listen,/g/glee/listen_20889265.html,Listen \rTo the song here in my heart \rA me...,"[glee club, hollywood, post-teen pop, viral pop]",0,0,1,0,0,0,0,listen to the song here in my heart melodi sta...
19557,Hank Snow,Just Keep A Movin',/h/hank+snow/just+keep+a+movin_20797171.html,Just keep a movin' wherever you were movin' \...,"[canadian country, canadian singer-songwriter,...",0,1,0,0,0,1,0,just keep movin wherev you be movin they tell ...


In [54]:
#Checking what the tokens column look like before removing stopwords
df_test['text_preprocessed'].head()

41483    when jump off have bucket full of thought when...
26062    in my dream you belong to me she doesn mean th...
135      be as young as feel alway be alway est la vie ...
18079    listen to the song here in my heart melodi sta...
19557    just keep movin wherev you be movin they tell ...
Name: text_preprocessed, dtype: object

In [55]:
#safety
df_test2 = df_test

## Tokenization

In [56]:
#w+ for whitespace
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

#creating a new column called tokens
df_test['tokens'] = df_test['text_preprocessed'].map(tokenizer.tokenize)

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [57]:
df_test.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens
41483,Phish,Back On The Train,/p/phish/back+on+the+train_20152042.html,"When I jumped off, I had a bucket full of thou...","[blues rock, jam band, roots rock]",1,0,0,0,0,0,0,when jump off have bucket full of thought when...,"[when, jump, off, have, bucket, full, of, thou..."
26062,Judds,In My Dreams,/j/judds/in+my+dreams_20076332.html,In my dreams \rYou belong to me \rShe doesn'...,"[country, country dawn, country rock]",1,0,0,0,0,1,0,in my dream you belong to me she doesn mean th...,"[in, my, dream, you, belong, to, me, she, does..."
135,Ace Of Base,C'est La Vie,/a/ace+of+base/cest+la+vie_10002096.html,"I am as young as I'm feeling \rI'm always 21,...","[bubblegum dance, dance pop, eurodance, europo...",1,0,1,0,0,0,0,be as young as feel alway be alway est la vie ...,"[be, as, young, as, feel, alway, be, alway, es..."
18079,Glee,Listen,/g/glee/listen_20889265.html,Listen \rTo the song here in my heart \rA me...,"[glee club, hollywood, post-teen pop, viral pop]",0,0,1,0,0,0,0,listen to the song here in my heart melodi sta...,"[listen, to, the, song, here, in, my, heart, m..."
19557,Hank Snow,Just Keep A Movin',/h/hank+snow/just+keep+a+movin_20797171.html,Just keep a movin' wherever you were movin' \...,"[canadian country, canadian singer-songwriter,...",0,1,0,0,0,1,0,just keep movin wherev you be movin they tell ...,"[just, keep, movin, wherev, you, be, movin, th..."


### Removing stop words
Reference for more info: https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

In [58]:
#removing stop words
df_test['tokens'] = df_test['tokens'].apply(lambda x: [item for item in x if item not in stop_words])

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [59]:
df_test.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens
41483,Phish,Back On The Train,/p/phish/back+on+the+train_20152042.html,"When I jumped off, I had a bucket full of thou...","[blues rock, jam band, roots rock]",1,0,0,0,0,0,0,when jump off have bucket full of thought when...,"[jump, bucket, full, thought, first, jump, hol..."
26062,Judds,In My Dreams,/j/judds/in+my+dreams_20076332.html,In my dreams \rYou belong to me \rShe doesn'...,"[country, country dawn, country rock]",1,0,0,0,0,1,0,in my dream you belong to me she doesn mean th...,"[dream, belong, mean, thing, never, wear, ring..."
135,Ace Of Base,C'est La Vie,/a/ace+of+base/cest+la+vie_10002096.html,"I am as young as I'm feeling \rI'm always 21,...","[bubblegum dance, dance pop, eurodance, europo...",1,0,1,0,0,0,0,be as young as feel alway be alway est la vie ...,"[young, feel, alway, alway, est, la, vie, est,..."
18079,Glee,Listen,/g/glee/listen_20889265.html,Listen \rTo the song here in my heart \rA me...,"[glee club, hollywood, post-teen pop, viral pop]",0,0,1,0,0,0,0,listen to the song here in my heart melodi sta...,"[listen, song, heart, melodi, start, complet, ..."
19557,Hank Snow,Just Keep A Movin',/h/hank+snow/just+keep+a+movin_20797171.html,Just keep a movin' wherever you were movin' \...,"[canadian country, canadian singer-songwriter,...",0,1,0,0,0,1,0,just keep movin wherev you be movin they tell ...,"[keep, movin, wherev, movin, tell, movin, keep..."


In [60]:
#checking out the data
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10225 entries, 41483 to 41301
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   artist             10225 non-null  object
 1   song               10225 non-null  object
 2   link               10225 non-null  object
 3   text               10225 non-null  object
 4   genres             10225 non-null  object
 5   rock               10225 non-null  int32 
 6   singer-songwriter  10225 non-null  int32 
 7   pop                10225 non-null  int32 
 8   metal              10225 non-null  int32 
 9   folk               10225 non-null  int32 
 10  country            10225 non-null  int32 
 11  hip hop / rap      10225 non-null  int32 
 12  text_preprocessed  10225 non-null  object
 13  tokens             10225 non-null  object
dtypes: int32(7), object(7)
memory usage: 918.7+ KB


In [61]:
#turning the stems back into untokenized structrure so that we can run count vector on it
df_test['stem_str'] = df_test['tokens'].map(lambda lst: ' '.join(lst))

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [62]:
df_test.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens,stem_str
41483,Phish,Back On The Train,/p/phish/back+on+the+train_20152042.html,"When I jumped off, I had a bucket full of thou...","[blues rock, jam band, roots rock]",1,0,0,0,0,0,0,when jump off have bucket full of thought when...,"[jump, bucket, full, thought, first, jump, hol...",jump bucket full thought first jump hold bucke...
26062,Judds,In My Dreams,/j/judds/in+my+dreams_20076332.html,In my dreams \rYou belong to me \rShe doesn'...,"[country, country dawn, country rock]",1,0,0,0,0,1,0,in my dream you belong to me she doesn mean th...,"[dream, belong, mean, thing, never, wear, ring...",dream belong mean thing never wear ring dream ...
135,Ace Of Base,C'est La Vie,/a/ace+of+base/cest+la+vie_10002096.html,"I am as young as I'm feeling \rI'm always 21,...","[bubblegum dance, dance pop, eurodance, europo...",1,0,1,0,0,0,0,be as young as feel alway be alway est la vie ...,"[young, feel, alway, alway, est, la, vie, est,...",young feel alway alway est la vie est la vie e...
18079,Glee,Listen,/g/glee/listen_20889265.html,Listen \rTo the song here in my heart \rA me...,"[glee club, hollywood, post-teen pop, viral pop]",0,0,1,0,0,0,0,listen to the song here in my heart melodi sta...,"[listen, song, heart, melodi, start, complet, ...",listen song heart melodi start complet listen ...
19557,Hank Snow,Just Keep A Movin',/h/hank+snow/just+keep+a+movin_20797171.html,Just keep a movin' wherever you were movin' \...,"[canadian country, canadian singer-songwriter,...",0,1,0,0,0,1,0,just keep movin wherev you be movin they tell ...,"[keep, movin, wherev, movin, tell, movin, keep...",keep movin wherev movin tell movin keep goin w...


### Vectorization

In [63]:
df_test['tokens2']=df_test.tokens.apply(lambda x:str(x).strip('[]'))
df_test.head()   

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens,stem_str,tokens2
41483,Phish,Back On The Train,/p/phish/back+on+the+train_20152042.html,"When I jumped off, I had a bucket full of thou...","[blues rock, jam band, roots rock]",1,0,0,0,0,0,0,when jump off have bucket full of thought when...,"[jump, bucket, full, thought, first, jump, hol...",jump bucket full thought first jump hold bucke...,"'jump', 'bucket', 'full', 'thought', 'first', ..."
26062,Judds,In My Dreams,/j/judds/in+my+dreams_20076332.html,In my dreams \rYou belong to me \rShe doesn'...,"[country, country dawn, country rock]",1,0,0,0,0,1,0,in my dream you belong to me she doesn mean th...,"[dream, belong, mean, thing, never, wear, ring...",dream belong mean thing never wear ring dream ...,"'dream', 'belong', 'mean', 'thing', 'never', '..."
135,Ace Of Base,C'est La Vie,/a/ace+of+base/cest+la+vie_10002096.html,"I am as young as I'm feeling \rI'm always 21,...","[bubblegum dance, dance pop, eurodance, europo...",1,0,1,0,0,0,0,be as young as feel alway be alway est la vie ...,"[young, feel, alway, alway, est, la, vie, est,...",young feel alway alway est la vie est la vie e...,"'young', 'feel', 'alway', 'alway', 'est', 'la'..."
18079,Glee,Listen,/g/glee/listen_20889265.html,Listen \rTo the song here in my heart \rA me...,"[glee club, hollywood, post-teen pop, viral pop]",0,0,1,0,0,0,0,listen to the song here in my heart melodi sta...,"[listen, song, heart, melodi, start, complet, ...",listen song heart melodi start complet listen ...,"'listen', 'song', 'heart', 'melodi', 'start', ..."
19557,Hank Snow,Just Keep A Movin',/h/hank+snow/just+keep+a+movin_20797171.html,Just keep a movin' wherever you were movin' \...,"[canadian country, canadian singer-songwriter,...",0,1,0,0,0,1,0,just keep movin wherev you be movin they tell ...,"[keep, movin, wherev, movin, tell, movin, keep...",keep movin wherev movin tell movin keep goin w...,"'keep', 'movin', 'wherev', 'movin', 'tell', 'm..."


## Doc2Vec

In [64]:
test_tokens_documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df_test.tokens)]

In [65]:
test_tokens_documents[1]

TaggedDocument(words=['dream', 'belong', 'mean', 'thing', 'never', 'wear', 'ring', 'dream', 'greatest', 'wish', 'come', 'true', 'arm', 'hold', 'dream', 'hear', 'laughter', 'tast', 'lip', 'feel', 'trembl', 'fingertip', 'night', 'mine', 'morn', 'find', 'go', 'dream', 'soul', 'wide', 'awak', 'crave', 'love', 'make', 'dream', 'dream', 'hide', 'thi', 'love', 'steal', 'til', 'feel', 'heart', 'feel', 'dream', 'dream', 'dream', 'dream'], tags=[1])

In [66]:
model_test = gensim.models.doc2vec.Doc2Vec(test_tokens_documents,
                                            vector_size = 50, 
                                            min_count = 2, 
                                            epochs = 20) #Takes 5min

In [67]:
model_test.train(test_tokens_documents, 
                  total_examples = model_test.corpus_count, 
                  epochs = model_test.epochs) #Takes 5min

In [68]:
model_test.infer_vector(test_tokens_documents[0].words, steps=20) #This changes every time. But within a certain range.

array([ 0.8138872 , -1.7641448 , -0.22665267,  1.9297285 ,  0.7225096 ,
        0.5510254 , -0.05726178,  1.0216082 , -0.13309771,  1.6064451 ,
       -0.8401656 , -1.250226  ,  1.4324117 ,  1.1221242 , -0.05193274,
       -0.43993372,  0.6590731 , -1.0130196 ,  0.11711044,  0.88469726,
       -0.01083186, -0.0249809 ,  0.16842951,  1.3415606 ,  0.00861675,
        0.8624664 , -0.19954541, -0.749401  ,  1.0246379 ,  0.67150587,
       -0.04709282, -0.58201206,  1.5655787 , -0.03664024,  0.91448194,
        0.27599388, -0.12488215, -0.7240846 ,  0.5717828 , -0.5992689 ,
        0.21970074, -2.0817482 ,  1.3465855 ,  1.3254272 ,  1.8047976 ,
        1.5850012 , -0.1469089 ,  0.6265584 , -0.87075365,  0.42316055],
      dtype=float32)

In [69]:
def vector_for_learning(model, input_docs):
    sents = input_docs
    targets, feature_vectors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, feature_vectors

In [70]:
target_test, veclyrics_doc2vec_test = vector_for_learning(model_test, test_tokens_documents) #Takes 5min

In [71]:
veclyrics_doc2vec_test[0]

array([ 0.80030984, -1.8968158 , -0.20183174,  2.038949  ,  0.6700128 ,
        0.80359936, -0.26136535,  0.8038724 , -0.01780331,  1.6143647 ,
       -0.66266567, -1.3268974 ,  1.4345216 ,  1.2054638 ,  0.06268109,
       -0.29674092,  0.84357643, -0.9477247 ,  0.10439444,  0.8167453 ,
       -0.02420871, -0.0391538 ,  0.11021158,  1.4326952 ,  0.01581643,
        1.0310534 , -0.2578711 , -0.6832181 ,  1.2554826 ,  0.5251921 ,
        0.09430447, -0.37911037,  1.4898267 , -0.3236018 ,  0.9216325 ,
        0.07969686, -0.20032321, -0.287201  ,  0.6446327 , -0.607193  ,
        0.2697619 , -2.0968423 ,  1.360646  ,  1.39385   ,  1.6997437 ,
        1.6787548 , -0.0832739 ,  0.7899511 , -0.6758349 ,  0.26415417],
      dtype=float32)

In [72]:
type(veclyrics_doc2vec_test)

tuple

In [73]:
len(veclyrics_doc2vec_test)

10225

## Proof of Concept of Doc2Vec for Prediction

In [74]:
clf = LogisticRegression().fit(veclyrics_doc2vec_train, df_train['rock'])
y_pred = clf.predict(veclyrics_doc2vec_test)

In [75]:
print('Logistic Regression Accuracy:', accuracy_score(df_test['rock'], y_pred))

Logistic Regression Accuracy: 0.7640097799511002


In [76]:
print(classification_report(df_test['rock'], y_pred))

              precision    recall  f1-score   support

           0       0.56      0.09      0.15      2459
           1       0.77      0.98      0.86      7766

    accuracy                           0.76     10225
   macro avg       0.67      0.53      0.51     10225
weighted avg       0.72      0.76      0.69     10225



### Note
Unlike with TF-IDF, the vectorized output using Doc2Vec is a tuple (nd-array) instead of a spare matrix

## Save vectorized lyrics and responses for train and test sets

#### Pickle vectorized lyrics

In [77]:
pickle.dump(veclyrics_doc2vec_train, open( "veclyrics_doc2vec_train.pkl", "wb" ))

In [78]:
pickle.dump(veclyrics_doc2vec_test, open( "veclyrics_doc2vec_test.pkl", "wb" ))

#### Format and pickle genre response

In [79]:
df_train = df_train.drop(['link','text','text_preprocessed','stem_str','tokens'], axis = 1)
df_test = df_test.drop(['link','text','text_preprocessed','stem_str','tokens'], axis = 1)

In [80]:
df_train.rename(columns={'tokens2':'tokenstring'}, inplace=True)
df_test.rename(columns={'tokens2':'tokenstring'}, inplace=True)

In [81]:
df_train.head()

,artist,song,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,tokenstring
40373,Paul Simon,Baby Driver,"[classic rock, folk, folk rock, mellow gold, p...",1,1,0,0,1,0,0,"'daddi', 'famili', 'bassman', 'mamma', 'engin'..."
41147,Peter Gabriel,San Jacinto,"[album rock, art pop, art rock, classic rock, ...",1,0,1,0,0,0,0,"'thick', 'cloud', 'steam', 'rise', 'hiss', 'st..."
26951,Keith Urban,Sometimes Angels Can't Fly,"[australian country, australian pop, contempor...",0,0,1,0,0,1,0,"'somewher', 'tonight', 'somebodi', 'cryin', 'f..."
32586,Marilyn Manson,Apple Of Sodom,"[alternative metal, industrial, industrial met...",1,0,0,1,0,0,0,"'find', 'center', 'fruit', 'late', 'center', '..."
28650,Kiss,I've Had Enough,"[album rock, classic rock, glam rock, hard roc...",1,0,0,1,0,0,0,"'mani', 'time', 'lie', 'truth', 'eye', 'treat'..."


In [82]:
df_test.head()

,artist,song,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,tokenstring
41483,Phish,Back On The Train,"[blues rock, jam band, roots rock]",1,0,0,0,0,0,0,"'jump', 'bucket', 'full', 'thought', 'first', ..."
26062,Judds,In My Dreams,"[country, country dawn, country rock]",1,0,0,0,0,1,0,"'dream', 'belong', 'mean', 'thing', 'never', '..."
135,Ace Of Base,C'est La Vie,"[bubblegum dance, dance pop, eurodance, europo...",1,0,1,0,0,0,0,"'young', 'feel', 'alway', 'alway', 'est', 'la'..."
18079,Glee,Listen,"[glee club, hollywood, post-teen pop, viral pop]",0,0,1,0,0,0,0,"'listen', 'song', 'heart', 'melodi', 'start', ..."
19557,Hank Snow,Just Keep A Movin',"[canadian country, canadian singer-songwriter,...",0,1,0,0,0,1,0,"'keep', 'movin', 'wherev', 'movin', 'tell', 'm..."


In [83]:
df_train.to_pickle("response_doc2vec_train.pkl")
df_test.to_pickle("response_doc2vec_test.pkl")